# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_deterministic.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::100] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([i+(i-1)*2 for i in range(100)], path="collected_data/rl_deterministic.txt")

[ 0.00000000e+00  7.95731469e+08 -7.69891077e-02  4.00000469e-03
  5.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.50295370e-01 -2.41931634e-01  6.00000000e-01]
Read chunk # -1 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.47410892e-01 -8.79999531e-03
  2.29998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  9.86362318e-02  3.75821673e-01 -6.00000000e-01]
Read chunk # 2 out of 10000
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08 -9.01891077e-02  1.08000047e-02
  3.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.17973088e-02 -2.40776052e-01  6.00000000e-01]
Read chunk # 8 out of 10000
[ 0.00000000e+00  7.95731469e+08  9.46108923e-02 -1.43999953e-02
 -1.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.56237335e-02  2.60569314e-01 -6.0

[ 0.00000000e+00  7.95731469e+08  6.41089229e-03 -1.73999953e-02
 -3.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.61668716e-02  4.19583882e-02 -1.17001341e-01]
Read chunk # 119 out of 10000
[ 0.00000000e+00  7.95731469e+08 -3.10891077e-02 -1.15999953e-02
  3.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.40217605e-01 -1.03693925e-01  2.47887223e-01]
Read chunk # 122 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.37889108e-01  1.74000047e-02
 -7.70001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.82335679e-01 -3.14817554e-01  6.00000000e-01]
Read chunk # 125 out of 10000
[ 0.00000000e+00  7.95731469e+08 -5.74891077e-02  1.36000047e-02
 -1.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -9.55353808e-02 -1.41985478e-01  5.72816347e-01]
Read chunk # 128 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.33610892e-01 -4.79999531e-03
  8.29998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.48931602e-01  3.00628710e-

[ 0.00000000e+00  7.95731469e+08  3.58108923e-02 -5.99999531e-03
  9.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.75852264e-01  3.66308726e-02 -3.51685115e-01]
Read chunk # 239 out of 10000
[ 0.00000000e+00  7.95731469e+08  5.74108923e-02  1.22000047e-02
 -2.50001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.17673299e-01  1.68558974e-01 -4.92447039e-01]
Read chunk # 242 out of 10000
[ 0.00000000e+00  7.95731469e+08 -3.31891077e-02 -1.51999953e-02
 -8.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.69532235e-01 -3.38468363e-02  2.55673559e-01]
Read chunk # 245 out of 10000
[ 0.00000000e+00  7.95731469e+08  3.41089229e-03 -4.59999531e-03
 -8.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.14383487e-01  6.36648794e-02 -4.80582627e-02]
Read chunk # 248 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.21389108e-01 -1.21999953e-02
 -4.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.34282315e-02 -2.92415991e-

In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -4.1981587,
 'std': 2.9079776,
 'min': -17.483574,
 'max': 0.0,
 'histogram': (array([  10,    8,   87,  177,  257,  288,  264,  296,  391,  577,  692,
          856, 1316, 2161, 2870, 3192, 4331, 5409, 3746,  842], dtype=int64),
  array([-17.483574 , -16.609396 , -15.735216 , -14.861038 , -13.986859 ,
         -13.11268  , -12.238502 , -11.364323 , -10.490145 ,  -9.615966 ,
          -8.741787 ,  -7.867608 ,  -6.9934297,  -6.119251 ,  -5.2450724,
          -4.3708935,  -3.4967148,  -2.6225362,  -1.7483574,  -0.8741787,
           0.       ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [6]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [7]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.23601703744964858


In [9]:
%load_ext tensorboard
%tensorboard --logdir runs

In [10]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=5, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-07 22:06.12 [debug    ] RoundIterator is selected.
2022-04-07 22:06.12 [info     ] Directory is created at d3rlpy_logs\CQL_20220407220612
2022-04-07 22:06.12 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-07 22:06.12 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-07 22:06.12 [warning  ] Skip building models since they're already built.
2022-04-07 22:06.12 [info     ] Parameters are saved to d3rlpy_logs\CQL_20220407220612\params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/5:   0%|          | 0/8590 [00:00<?, ?it/s]

2022-04-07 22:50.17 [info     ] CQL_20220407220612: epoch=1 step=8590 epoch=1 metrics={'time_sample_batch': 0.0009770955140154909, 'time_algorithm_update': 0.29735887328182303, 'temp_loss': 0.3410345619932528, 'temp': 0.9888876052965802, 'alpha_loss': 8.785565134753224, 'alpha': 0.7231650749622596, 'critic_loss': 36.18224927438846, 'actor_loss': 24.133028367034033, 'time_step': 0.29970271362553375, 'td_error': 71.8260358979081, 'init_value': -48.7784538269043, 'ave_value': -48.87078756352829} step=8590
2022-04-07 22:50.17 [info     ] Model parameters are saved to d3rlpy_logs\CQL_20220407220612\model_8590.pt


Epoch 2/5:   0%|          | 0/8590 [00:00<?, ?it/s]

2022-04-07 23:40.17 [info     ] CQL_20220407220612: epoch=2 step=17180 epoch=2 metrics={'time_sample_batch': 0.0008257755439411914, 'time_algorithm_update': 0.33884366982474456, 'temp_loss': -0.00011581030248190526, 'temp': 1.2431983920276095, 'alpha_loss': 4.4406926053783256, 'alpha': 0.32912610182072427, 'critic_loss': 193.84284772006959, 'actor_loss': 61.46111132096634, 'time_step': 0.34103232834475145, 'td_error': 123.44744246701725, 'init_value': -60.01475143432617, 'ave_value': -60.1333289133089} step=17180
2022-04-07 23:40.17 [info     ] Model parameters are saved to d3rlpy_logs\CQL_20220407220612\model_17180.pt


...

Epoch 4/5:   0%|          | 0/8590 [00:00<?, ?it/s]

2022-04-08 01:18.56 [info     ] CQL_20220407220612: epoch=4 step=34360 epoch=4 metrics={'time_sample_batch': 0.0009392525458641186, 'time_algorithm_update': 0.3300192184581468, 'temp_loss': 0.03313248623246533, 'temp': 0.9043158824979496, 'alpha_loss': 0.5097132887270592, 'alpha': 0.07484302060620054, 'critic_loss': 248.88140230060873, 'actor_loss': 55.150059736427245, 'time_step': 0.3324029747626557, 'td_error': 122.52313128510093, 'init_value': -47.87778091430664, 'ave_value': -48.00197334582618} step=34360
2022-04-08 01:18.56 [info     ] Model parameters are saved to d3rlpy_logs\CQL_20220407220612\model_34360.pt


Epoch 5/5:   0%|          | 0/8590 [00:00<?, ?it/s]

2022-04-08 02:06.50 [info     ] CQL_20220407220612: epoch=5 step=42950 epoch=5 metrics={'time_sample_batch': 0.0009074546960513167, 'time_algorithm_update': 0.3238304439329296, 'temp_loss': 0.023459250246590964, 'temp': 0.7026218292876922, 'alpha_loss': 0.17984766460330648, 'alpha': 0.03568290511516982, 'critic_loss': 201.15485333642803, 'actor_loss': 41.53012437309736, 'time_step': 0.3261824154048915, 'td_error': 93.56227844641366, 'init_value': -33.98887252807617, 'ave_value': -34.097401090584675} step=42950
2022-04-08 02:06.50 [info     ] Model parameters are saved to d3rlpy_logs\CQL_20220407220612\model_42950.pt


[(1,
  {'time_sample_batch': 0.0009770955140154909,
   'time_algorithm_update': 0.29735887328182303,
   'temp_loss': 0.3410345619932528,
   'temp': 0.9888876052965802,
   'alpha_loss': 8.785565134753224,
   'alpha': 0.7231650749622596,
   'critic_loss': 36.18224927438846,
   'actor_loss': 24.133028367034033,
   'time_step': 0.29970271362553375,
   'td_error': 71.8260358979081,
   'init_value': -48.7784538269043,
   'ave_value': -48.87078756352829}),
 (2,
  {'time_sample_batch': 0.0008257755439411914,
   'time_algorithm_update': 0.33884366982474456,
   'temp_loss': -0.00011581030248190526,
   'temp': 1.2431983920276095,
   'alpha_loss': 4.4406926053783256,
   'alpha': 0.32912610182072427,
   'critic_loss': 193.84284772006959,
   'actor_loss': 61.46111132096634,
   'time_step': 0.34103232834475145,
   'td_error': 123.44744246701725,
   'init_value': -60.01475143432617,
   'ave_value': -60.1333289133089}),
 (3,
  {'time_sample_batch': 0.000880347724843497,
   'time_algorithm_update': 0.33

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [11]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_deterministic.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08 -7.69891077e-02  4.00000469e-03
  5.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.50295370e-01 -2.41931634e-01  6.00000000e-01]
Read chunk # 1 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.01089229e-03 -4.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21623335e-01 -2.86362315e-02 -8.00043364e-02]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01 -1.37999953e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.76352555e-01 -3.26280816e-01  6.00

[ 0.00000000e+00  7.95731469e+08 -1.12689108e-01 -7.79999531e-03
 -9.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.21066700e-01 -2.39099259e-01  6.00000000e-01]
Read chunk # 81 out of 10000
[ 0.00000000e+00  7.95731469e+08  6.10108923e-02  1.32000047e-02
  7.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.44237123e-01  1.16216092e-01 -5.22459650e-01]
Read chunk # 83 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.99891077e-02 -1.91999953e-02
  3.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.62080602e-01 -7.23826969e-02  1.20921962e-01]
Read chunk # 85 out of 10000
[ 0.00000000e+00  7.95731469e+08  6.01108923e-02  4.20000469e-03
  6.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.55040552e-01  1.19634499e-01 -5.43152828e-01]
Read chunk # 87 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.41410892e-01 -1.71999953e-02
  4.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.84606127e-01  3.47696536e-01 -

[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  2.20000469e-03
 -3.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -9.45911398e-02 -2.84914456e-01  6.00000000e-01]
Read chunk # 161 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.09891077e-02 -8.79999531e-03
  5.79998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.95571791e-01 -6.59640509e-02  7.13476936e-02]
Read chunk # 163 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.33891077e-02  1.40000047e-02
  8.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.93683309e-01 -9.29396023e-02  1.66975975e-01]
Read chunk # 165 out of 10000
[ 0.00000000e+00  7.95731469e+08  4.91089229e-03  6.00004692e-04
  8.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21954598e-01 -3.87056001e-02 -4.51495231e-02]
Read chunk # 167 out of 10000
[ 0.00000000e+00  7.95731469e+08 -8.95891077e-02  1.60000469e-03
 -6.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.80835956e-01 -1.96007398e-

Epoch 1/100:   0%|          | 0/21992 [00:00<?, ?it/s]

C:\Users\Aero\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Aero\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

2022-04-08 10:09.27 [info     ] Model parameters are saved to d3rlpy_logs\FQE_20220408020741\model_2177208.pt


Epoch 100/100:   0%|          | 0/21992 [00:00<?, ?it/s]

2022-04-08 10:14.37 [info     ] FQE_20220408020741: epoch=100 step=2199200 epoch=100 metrics={'time_sample_batch': 0.0003767586721425145, 'time_algorithm_update': 0.011553301285205646, 'loss': 0.01729562495076955, 'time_step': 0.01263644972598609, 'init_value': -1.2745429277420044, 'ave_value': -1.2739250281759056, 'soft_opc': nan} step=2199200


2022-04-08 10:14.37 [info     ] Model parameters are saved to d3rlpy_logs\FQE_20220408020741\model_2199200.pt


[(1,
  {'time_sample_batch': 0.00042819874899827076,
   'time_algorithm_update': 0.0068035466036132225,
   'loss': 0.008132032031880457,
   'time_step': 0.007887866907702571,
   'init_value': -1.04853093624115,
   'ave_value': -1.0482471513921092,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.0003912838730824215,
   'time_algorithm_update': 0.006498671653445047,
   'loss': 0.014115804965008364,
   'time_step': 0.0074729861867085245,
   'init_value': -1.1801666021347046,
   'ave_value': -1.1798491936532454,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.0003805624306874086,
   'time_algorithm_update': 0.007124566942962918,
   'loss': 0.013537545082770205,
   'time_step': 0.008145781149210258,
   'init_value': -1.1299998760223389,
   'ave_value': -1.1296853448110773,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.0003992889676800031,
   'time_algorithm_update': 0.007742372349246106,
   'loss': 0.01322349580627041,
   'time_step': 0.008771122661925698,
   'init_value': 

In [12]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_stochastic.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08 -1.02189108e-01 -1.57999953e-02
 -9.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.03154890e-01 -2.08087043e-01  6.00000000e-01]
Read chunk # 1 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.39310892e-01  1.82000047e-02
 -1.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.50230117e-02  3.69851546e-01 -6.00000000e-01]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  1.64000047e-02
 -8.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08831172e-01 -2.48178665e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.28589108e-01  1.20000047e-02
  1.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.75212759e-03 -3.52719043e-01  6.00000000e-01]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01  7.00000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.65974295e-01 -2.19295880e-01  6.00

[ 0.00000000e+00  7.95731469e+08 -6.18910771e-03 -1.21999953e-02
  6.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.20148033e-01 -5.58381024e-02  1.60780066e-02]
Read chunk # 81 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.40289108e-01 -7.99995308e-04
 -7.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.90695143e-01 -3.25680916e-01  6.00000000e-01]
Read chunk # 83 out of 10000
[ 0.00000000e+00  7.95731469e+08 -8.26891077e-02  1.42000047e-02
 -8.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.83479718e-01 -1.65500280e-01  6.00000000e-01]
Read chunk # 85 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.60108923e-02 -6.39999531e-03
  6.89998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.16509328e-01  1.57204751e-01 -6.00000000e-01]
Read chunk # 87 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.84108923e-02  1.40000047e-02
  8.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.71526610e-01  1.55178362e-01 -

[ 0.00000000e+00  7.95731469e+08  3.37108923e-02 -2.99999531e-03
 -5.90001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.51495751e-01  1.27746051e-01 -3.22630690e-01]
Read chunk # 161 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.39389108e-01  4.00004692e-04
 -8.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.34233329e-01 -3.14276109e-01  6.00000000e-01]
Read chunk # 163 out of 10000
[ 0.00000000e+00  7.95731469e+08 -8.59891077e-02  1.18000047e-02
  6.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.35947142e-01 -2.69618370e-01  6.00000000e-01]
Read chunk # 165 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.08789108e-01 -5.59999531e-03
 -6.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.55021771e-01 -2.46178506e-01  6.00000000e-01]
Read chunk # 167 out of 10000
[ 0.00000000e+00  7.95731469e+08  6.71089229e-03 -1.21999953e-02
 -3.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.46540079e-02  3.75964814e-

Epoch 1/100:   0%|          | 0/21992 [00:00<?, ?it/s]

C:\Users\Aero\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Aero\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

2022-04-08 18:06.57 [info     ] Model parameters are saved to d3rlpy_logs\FQE_20220408101531\model_2177208.pt


Epoch 100/100:   0%|          | 0/21992 [00:00<?, ?it/s]

2022-04-08 18:12.02 [info     ] FQE_20220408101531: epoch=100 step=2199200 epoch=100 metrics={'time_sample_batch': 0.00040322245143031415, 'time_algorithm_update': 0.011376602665299978, 'loss': 0.021428368828787916, 'time_step': 0.012449288929796687, 'init_value': -1.385913610458374, 'ave_value': -1.3852465892698747, 'soft_opc': nan} step=2199200


2022-04-08 18:12.02 [info     ] Model parameters are saved to d3rlpy_logs\FQE_20220408101531\model_2199200.pt


[(1,
  {'time_sample_batch': 0.0004048699593708792,
   'time_algorithm_update': 0.006098231745876369,
   'loss': 0.008039890150641332,
   'time_step': 0.007181064936844988,
   'init_value': -1.154395341873169,
   'ave_value': -1.1544075110241856,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00037823652692082235,
   'time_algorithm_update': 0.006257258098140202,
   'loss': 0.013434912411280527,
   'time_step': 0.007295431571251352,
   'init_value': -1.180904746055603,
   'ave_value': -1.1811056208214301,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00041217648276158705,
   'time_algorithm_update': 0.006883125384966648,
   'loss': 0.014252102441193727,
   'time_step': 0.007972539425243157,
   'init_value': -1.1604664325714111,
   'ave_value': -1.1606499106798156,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.000401743122255555,
   'time_algorithm_update': 0.007437083332180413,
   'loss': 0.015185871014634358,
   'time_step': 0.008545193673481114,
   'init_value': -